3 phase imbalance power flow

In [55]:
import pandapower as pp

# empty grid
grid = pp.create_empty_network() 

# define unsymmetric line as standard type (component library)
unsymmetric_line = {"r_ohm_per_km": 0.1941, "x_ohm_per_km": 0.07476991, "c_nf_per_km": 1160., "max_i_ka": 0.421,
                    "endtemp_degree": 70.0, "r0_ohm_per_km": 0.7766,"x0_ohm_per_km": 0.2990796,"c0_nf_per_km":  500.}
pp.create_std_type(grid, unsymmetric_line, name="unsymmetric_line",element = "line")

# define unsymmetric trafo as standard type (component library)
unsymmetric_trafo = {"sn_mva": 1.6, "vn_hv_kv": 20,"vn_lv_kv": 0.4, "vk_percent": 6, "vkr_percent": 0.78125,
                     "pfe_kw": 0.0, "i0_percent": 0.0, "shift_degree": 0, "vector_group": "Dyn", "tap_side": "lv", "tap_neutral": 0,
                     "tap_min": -2, "tap_max": 2, "tap_step_degree": 0, "tap_step_percent": 2.5, "tap_phase_shifter": False,
                     "vk0_percent": 6, "vkr0_percent": 0.78125, "mag0_percent": 100, "mag0_rx": 0., "si0_hv_partial": 0.9}


unsymmetric_trafo_x = {"sn_mva": 1.6, "vn_hv_kv": 20,"vn_lv_kv": 0.4}

pp.create_std_type(grid,unsymmetric_trafo, name='unsymmetric_trafo', element="trafo")
pp.create_std_type(grid,unsymmetric_trafo_x, name='unsymmetric_trafo_x', element="trafo")
# create buses
b1 = pp.create_bus(grid, vn_kv=20.0, name="Bus 0")
b2 = pp.create_bus(grid, vn_kv=0.4, name="Bus 1")
b3 = pp.create_bus(grid, vn_kv=0.4, name="Bus 2")
#b4 = pp.create_bus(grid, vn_kv=0.4, name = "Bus 4")
#b32 = pp.create_bus(grid, vn_kv=0.4, name="Bus 3.2")

# create network feeder
#pp.create_ext_grid(grid, bus=b1, vm_pu=1.02, name="Grid Connection", s_sc_max_mva=1000, rx_max=1, x0x_max=1, r0x0_max=1)

pp.create_ext_grid(grid, bus=b1, vm_pu=1.02, name="Grid Connection", s_sc_max_mva=1000)

# create loads
pp.create_asymmetric_load(grid, bus=b3, p_a_mw=0.05, p_b_mw=0.05, p_c_mw=0.05, q_a_mvar=0, q_b_mvar=0,
                          q_c_mvar=0, sn_mva=0, name=None, scaling=1.0, index=None, in_service=True, type='wye')
# pp.create_asymmetric_load(grid, bus=b31, p_a_mw=0.0, p_b_mw=0.0, p_c_mw=0.8, q_a_mvar=0, q_b_mvar=0, q_c_mvar=0, sn_mva=0.06, name=None, scaling=1.0, index=None, in_service=True, type='delta')

# create trafos
#t1 = pp.create_transformer(grid, hv_bus=b1, lv_bus=b2, std_type="unsymmetric_trafo", name="Trafo")
t1 = pp.create_transformer(grid, hv_bus=b1, lv_bus=b2, std_type="unsymmetric_trafo_x", name="Trafo")
# create lines
l1 = pp.create_line(grid, from_bus=b2, to_bus=b3, length_km=0.1, name="Line",std_type="unsymmetric_line") 
#l2 = pp.create_line(grid, from_bus=b2, to_bus=b32, length_km=0.1, name="Line",std_type="unsymmetric_line") 
# l3 = pp.create_line(grid, from_bus=b31, to_bus=b32, length_km=0.1, name="Line",std_type="unsymmetric_line") 

#l2 = pp.create_line(grid, b2 , b4, std_type="NAYY 4x50 SE",length_km=0.1, name="line")

# adds the zero sequence parameters of the standard types to the grid model
pp.add_zero_impedance_parameters(grid)

# run three-phase power flow
pp.runpp_3ph(grid, calculate_voltage_angles=True, init='auto', max_iteration='auto',
             tolerance_mva=1e-08, trafo_model='t', trafo_loading='current',
             enforce_q_lims=False, numba=True, recycle=None, check_connectivity=True, switch_rx_ratio=2.0, delta_q=0, v_debug=False)

UserWarning: vk_percent is required as trafo type parameter

In [54]:
print(grid.res_ext_grid_3ph)

     p_a_mw  q_a_mvar    p_b_mw  q_b_mvar    p_c_mw  q_c_mvar
0  0.050945  0.000626  0.050945  0.000626  0.050945  0.000626


In [56]:
w = [14.82,23.95,12.91,64.64,30.56,44.73,21.77,34.59,154.50,136.52,5.68,14.42,4.69,106.95,48.32,154.50,49.23,4.28,
     2.28,2.28,962.08,17.51,16.40,2.28,104.32,34.03,18.34,23.47,23.54]

sum(w)

2133.59

In [57]:
z = [5.10,0.74,55.15,23.08,10.73,49.76,22.01,11.77,3.23,147.04,12.92]

sum(z)

341.53000000000003

In [58]:
c = [15.50,2249]

sum(c)

2264.5

In [59]:
2978/9645

0.3087610160705028